#Crawling PTA Trunojoyo

In [1]:
import requests
import csv
import pandas as pd
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings("ignore")
headers = {
        'Access-Control-Allow-Origin': '*',
        'Access-Control-Allow-Methods': 'GET',
        'Access-Control-Allow-Headers': 'Content-Type',
        'accept': '/',
        'accept-encoding': 'gzip, deflate',
        'accept-language': 'en,mr;q=0.9',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'}

In [2]:
list_judul = []
list_penulis = []
list_pembimbing1 = []
list_pembimbing2 = []
list_abstak = []
url = f"https://pta.trunojoyo.ac.id/c_search/byprod/10/"
req = requests.get(url,headers=headers,timeout=5,verify=False)
for i in range(2, 171):
    soup = BeautifulSoup(req.content, 'html.parser')
    results = soup.find(id="content_journal")
    elements = results.find_all("li", attrs={"data-id":"id-1"})
#     print(elements)
    for element in elements:
        info = element.find_all("div")[0]
        info_nama = info.find_all("div")
#         print(info_nama)
        link_abstrak = element.find_all("div")[-1].find("a")["href"]
#         ekstrak info
        judul = info.find("a", class_="title").text
        penulis, pembimbing1, pembimbing2 = [nama.text.split(":")[1].strip() for nama in info_nama]

#         ekstrak abstrak

#         print(link_abstrak)
        req = requests.get(link_abstrak,headers=headers,timeout=5,verify=False)
        soup_abstrak =  BeautifulSoup(req.content, 'html.parser')
        results_abstrak = soup_abstrak.find_all("p")

        abstrak = results_abstrak[0].text

        list_judul.append(judul)
        list_penulis.append(penulis)
        list_pembimbing1.append(pembimbing1)
        list_pembimbing2.append(pembimbing2)
        list_abstak.append(abstrak)

    url = f"https://pta.trunojoyo.ac.id/c_search/byprod/10/{i}"
    req = requests.get(url,headers=headers,timeout=50,verify=False)
    if(req.status_code == 404):
        break

In [3]:
df = pd.DataFrame({
    "Judul" : list_judul,
    "Penulis" : list_penulis,
    "Pembimbing I" : list_pembimbing1,
    "Pembimbing II" : list_pembimbing2,
    "Abstrak" : list_abstak
})
df
#df.to_csv("ptaUtm.csv", index=True)

,Judul,Penulis,Pembimbing I,Pembimbing II,Abstrak
0,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE \...,A.Ubaidillah S.Kom,Budi Setyono M.T,Hermawan S.T,Sistem informasi akademik (SIAKAD) merupaka...
1,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,"M. Basith Ardianto,","Drs. Budi Soesilo, MT","Koko Joni, ST",Berjalannya koneksi jaringan komputer dengan l...
2,RANCANG BANGUN APLIKASI PROXY SERVER UNTUK\r\n...,"Akhmad Suyandi, S.Kom","Drs. Budi Soesilo, M.T","Hermawan, ST, MT",Web server adalah sebuah perangkat lunak serve...
3,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,Heri Supriyanto,"Mulaab, S.Si., M.Kom","Firli Irhamni, ST., M.Kom",Penjadwalan kuliah di Perguruan Tinggi me...
4,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,Septian Rahman Hakim,"Arik Kurniawati, S.Kom., M.T.","Haryanto, S.T., M.T.",Seiring perkembangan teknologi yang ada diduni...
...,...,...,...,...,...
840,PENERAPAN NON-NEGATIVE MATRIX FACTORIZATION DA...,Moh. Nurun Fitriantama,"Dr. Noor Ifada, S.T., M.ISD.","Moch. Kautsar Sophan, S.Kom., M.MT.",Dengan berkembangnya teknologi yang pesat dan ...
841,PENERAPAN METODE TECHNIQUE FOR OTHERS PREFEREN...,GILANG ABDURRACHMAN PUTRA,"SRI HERAWATI, S.KOM., M.KOM","EKA MALA SARI R. S.KOM., M.KOM",Perkembangan teknologi internet yang sangat pe...
842,PEMANFAATAN METODE TECHNIQUE FOR ORDER PREFERE...,Abi Dzar Giffary,"Achmad Jauhari, S.T., M.Kom.","Sri Herawati, S.Kom., M.Kom.",ABSTRAK\n\nPenilaian kinerja karyawan merupaka...
843,ANALISIS MULTIPATH ROUTING BERBASIS ALGORITHMA...,Imam Achmad Al Abidulloh,"Dwi Kuswanto, S.Pd., M.T.","Iwan Santosa, S.T., M.T.",Algoritma routing pada jaringan SDN sangat men...
